In [45]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/sloberta"

In [47]:
from datasets import load_dataset

dataset = load_dataset("anzeo/Slovene_SuperGLUE_CB")

In [48]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [2]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
Y

In [123]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"


def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [124]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

In [125]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [126]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [127]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [3]:
from transformers import set_seed

set_seed(42)

In [129]:
from peft import LoHaConfig, get_peft_model, TaskType

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [130]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,772,547 || all params: 112,396,806 || trainable%: 1.5770


In [4]:
from transformers import TrainingArguments

new_model_name = "loha_fine_tuned_cb_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-3,
    weight_decay=0.02,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [132]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [133]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=400, training_loss=0.22283551572822036, metrics={'train_runtime': 43.586, 'train_samples_per_second': 73.418, 'train_steps_per_second': 9.177, 'total_flos': 187914496291704.0, 'train_loss': 0.22283551572822036, 'epoch': 28.571428571428573})

In [134]:
trainer.evaluate()

{'eval_loss': 2.2668612003326416,
 'eval_accuracy': 0.6363636363636364,
 'eval_f1': 0.5976511526750761,
 'eval_runtime': 0.1895,
 'eval_samples_per_second': 116.095,
 'eval_steps_per_second': 15.831,
 'epoch': 28.571428571428573}

In [135]:
trainer.push_to_hub(new_model_name)

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/7.10M [00:00<?, ?B/s]

events.out.tfevents.1716371684.DESKTOP-22QTFDR.2016.6:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

events.out.tfevents.1716371727.DESKTOP-22QTFDR.2016.7:   0%|          | 0.00/457 [00:00<?, ?B/s]

Upload 12 LFS files:   0%|          | 0/12 [00:00<?, ?it/s]

...

events.out.tfevents.1716373729.DESKTOP-22QTFDR.2016.13:   0%|          | 0.00/457 [00:00<?, ?B/s]

events.out.tfevents.1716373815.DESKTOP-22QTFDR.2016.14:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

events.out.tfevents.1716373858.DESKTOP-22QTFDR.2016.15:   0%|          | 0.00/457 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anzeo/loha_fine_tuned_cb_sloberta/commit/0cc9a64ef9326daedbe22a6d3e2991a4528244fd', commit_message='loha_fine_tuned_cb_sloberta', commit_description='', oid='0cc9a64ef9326daedbe22a6d3e2991a4528244fd', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
# Example
hypothesis = "Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [6]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification
import torch

# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)

adapter_name = "anzeo/" + new_model_name

config = PeftConfig.from_pretrained(adapter_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, adapter_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_name)

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

outputs = model(**inputs, labels=label)
logits = outputs.logits

In [8]:
# Print prediction
logits.argmax().item()

2